# Text-To-Text LLM Server

**important: Select venv Python Interpreter before you start**

This repository is designed to be used with Visual Studio Code and Docker DevContainer.

![dev-container](../img/dev-container.png)

## 1. Setup

**Instructions:**

a) Download model

```bash
huggingface-cli download bartowski/Mistral-7B-Instruct-v0.3-exl2 \
    --revision 1a09a351a5fb5a356102bfca2d26507cdab11111 \
    --local-dir ~/.gai/models/exllamav2-mistral7b \
    --local-dir-use-symlinks False
```

or

```bash
huggingface-cli download bartowski/dolphin-2.9-llama3-8b-exl2 \
    --revision 6521bd8b0f793a038f85d445316c94cdd0957d8e \
    --branch 4_25 \
    --local-dir ~/.gai/models/exllamav2-mistral7b \
    --local-dir-use-symlinks False
```

b) Create gai.yml in ~/.gai

```yaml
gen:
    default:
        ttt: "ttt-exllamav2-dolphin"
    ttt-exllamav2-mistral7b:
        type: "ttt"
        generator_name: "exllamav2-mistral7b"
        engine: "gai.ttt.server.GaiExLlamaV2"
        model_path: "models/exllamav2-mistral7b"
        model_basename: "model"
        max_seq_len: 8192
        prompt_format: "mistral"
        hyperparameters:
            {
                "temperature": 0.85,
                "top_p": 0.8,
                "top_k": 50,
                "max_tokens": 1000,
            }
        tool_choice: "auto"
        max_retries: 5
        stop_conditions: ["<|im_end|>", "</s>", "[/INST]"]
        no_flash_attn: true
        seed: null
        decode_special_tokens: false
        module_name: "gai.ttt.server.gai_exllamav2"
        class_name: "GaiExLlamav2"
        init_args: []
        init_kwargs: {}
    ttt-exllamav2-dolphin:
        type: "ttt"
        generator_name: "exllamav2-dolphin"
        engine: "gai.ttt.server.GaiExLlamaV2"
        model_path: "models/exllamav2-dolphin"
        model_basename: "model"
        max_seq_len: 8192
        prompt_format: "mistral"
        hyperparameters:
            {
                "temperature": 0.85,
                "top_p": 0.8,
                "top_k": 50,
                "max_tokens": 1000,
            }
        tool_choice: "auto"
        max_retries: 5
        stop_conditions: ["<|im_end|>", "</s>", "[/INST]"]
        no_flash_attn: true
        seed: null
        decode_special_tokens: false
        module_name: "gai.ttt.server.gai_exllamav2"
        class_name: "GaiExLlamav2"
        init_args: []
        init_kwargs: {}
```

---

## 2. Smoke Test

In [1]:
# check .gairc
import os
gairc=None
with open(os.path.expanduser("~/.gairc"),"r") as f:
    gairc = f.read()
print(gairc)

# check ~/.gairc (if docker created .gairc)
import json
jsoned=json.loads(gairc)
assert jsoned["app_dir"]=="~/.gai"

# check ~/.gai (if docker created the mount point)
assert os.path.exists(os.path.expanduser("~/.gai"))

# Initiate
from gai.lib.server.singleton_host import SingletonHost
from gai.lib.common.utils import free_mem
from rich.console import Console
console=Console()

config = {
    "type": "ttt",
    "generator_name": "exllamav2-dolphin",
    "engine": "gai.ttt.server.GaiExLlamaV2",
    "model_path": "models/exllamav2-dolphin",
    "model_basename": "model",
    "max_seq_len": 8192,
    "prompt_format": "mistral",
    "hyperparameters": {
        "temperature": 0.85,
        "top_p": 0.8,
        "top_k": 50,
        "max_tokens": 1000,
    },
    "tool_choice": "auto",
    "max_retries": 5,
    "stop_conditions": ["<|im_end|>", "</s>", "[/INST]"],
    "no_flash_attn":True,
    "seed": None,
    "decode_special_tokens": False,
    "module_name": "gai.ttt.server.gai_exllamav2",
    "class_name": "GaiExLlamav2",
    "init_args": [],
    "init_kwargs": {}
}

# before loading
free_mem()
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:

        # after loading
        free_mem()
except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()

{"app_dir": "~/.gai"}


Free memory: 1.17 GB

                      Performance load_config                      
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 5.32 seconds │ 5.32 seconds │
│ CPU Memory        │          - MB │     21.16 MB │     21.16 MB │
│ CUDA Memory       │       1.18 GB │     -4.64 GB │      5.82 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

                      Performance load_model                       
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 0.01 seconds │ 0.01 seconds │
│ CPU Memory        │          - MB │      0.22 MB │      0.22 MB │
│ CUDA Memory       │       5.82 GB │      0.00 GB │      5.82 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

                       Performance load_cache                        
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃  Change Value ┃   Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 24.64 seconds │ 24.64 seconds │
│ CPU Memory        │          - MB │      24.18 MB │      24.18 MB │
│ CUDA Memory       │       5.82 GB │       4.61 GB │       1.21 GB │
└───────────────────┴───────────────┴───────────────┴───────────────┘

                    Performance load_tokenizer                     
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 2.97 seconds │ 2.97 seconds │
│ CPU Memory        │          - MB │     49.66 MB │     49.66 MB │
│ CUDA Memory       │       1.21 GB │      0.00 GB │      1.21 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

Free memory: 1.21 GB

Free memory: 5.71 GB

---

## 3. Integration Test

### Startup

In [2]:
host = SingletonHost.GetInstanceFromConfig(config, verbose=False)
host.load()
generator = host.generator
free_mem()

Free memory: 1.21 GB

1.2125511169433594

### a) Test streaming

In [3]:
response = host.generator.create(
    messages=[{"role":"user","content":"Tell me a one paragraph story"},
                {"role":"assistant","content":""}],
    stream=True)
for message in response:
    if message.choices[0].delta.content:
        print(message.choices[0].delta.content, end="", flush=True)
   

A young boy named Timmy was always curious and adventurous. One day, while exploring the woods near his home, he stumbled upon a magical tree that bore fruits of all colors and flavors. As he plucked a fruit from each branch, the tree began to shrink, and Timmy realized that he had to choose just one fruit to keep. After much thought, he chose a golden apple, which turned out to be the most delicious fruit he had ever tasted. From that day forward, every time he felt sad or lonely, he would take a bite of the golden apple, and it would fill him with joy and contentment. And so, the magical tree taught Timmy the lesson of moderation and the happiness that comes from simple pleasures.

### b) Test generation

In [4]:

response = host.generator.create(
    messages=[{"role":"user","content":"Tell me a one paragraph story"},
                {"role":"assistant","content":""}],
    stream=False)
print(response.choices[0].message.content)


A young boy named Timmy had always dreamed of owning a dog. One day, while walking home from school, he found a small, scruffy puppy abandoned on the side of the road. Despite being dirty and scared, Timmy took the puppy home and cared for it, and soon they became inseparable best friends. The puppy, whom Timmy named Buddy, grew up to be a loyal and loving companion, proving that sometimes, the most unlikely encounters can lead to the deepest connections.


### c) Test Tool Calling

In [5]:
messages = [
    {"role":"user","content":"What is the current time in Singapore?"},
    {"role":"assistant","content":""}
]
tool_choice="required"
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
response = host.generator.create(
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    stream=False)
print(response)


ChatCompletion(id='chatcmpl-27522c91-0921-44b1-a686-71e5df96aa93', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2967e378-5896-4bdf-896b-fbbe7b8360a8', function=Function(arguments='{"search_query": "current time in Singapore"}', name='google'), type='function')]))], created=1733495854, model='exllamav2-mistral7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=418, total_tokens=468, completion_tokens_details=None, prompt_tokens_details=None))


### d) Test Structured Output

In [6]:
# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""
response = host.generator.create(messages=[{'role':'user','content':text},{'role':'assistant','content':''}], 
    json_schema=Book.schema(),
    stream=False
    )
print(response)


/tmp/ipykernel_5744/2228101676.py:18: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json_schema=Book.schema(),


ChatCompletion(id='chatcmpl-2b28e476-b27c-41eb-aae8-43469aab9455', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n "title": "Foundation",\n "summary": "Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.",\n "author": "Isaac Asimov",\n "published_year": 1951\n }', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733495863, model='exllamav2-mistral7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=144, 

### Teardown

In [7]:
del host.generator.model
del host.generator.cache
del host.generator.tokenizer
del host.generator
import gc,torch
gc.collect()
torch.cuda.empty_cache()
free_mem()

Free memory: 1.44 GB

1.4351615905761719

---

## 4. API Test

**Instructions**:

a) Open Debug Icon and select **Python Debugger: gai-ttt server (dolphin)**

b) Press `F5` to start the API server.

c) Wait for the server to start.


**Tests**:

Run the following cells to test the API.

### a) Test Generating

In [8]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story.\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"tool_choice\": \"none\"}"
        

CalledProcessError: Command 'b'curl -X POST \\\n    http://localhost:12031/gen/v1/chat/completions \\\n    -H \'Content-Type: application/json\' \\\n    -s \\\n    -N \\\n    -d "{\\"model\\":\\"exllamav2-mistral7b\\", \\\n        \\"messages\\": [ \\\n            {\\"role\\": \\"user\\",\\"content\\": \\"Tell me a story.\\"}, \\\n            {\\"role\\": \\"assistant\\",\\"content\\": \\"\\"} \\\n        ],\\\n        \\"tool_choice\\": \\"none\\"}"\n        \n'' returned non-zero exit status 7.

### b) Test Streaming

In [5]:
import json
import httpx
import asyncio
from openai import ChatCompletion

json_payload = {
    "temperature": 0.2,
    "max_tokens": 50,
    "stream": "true",  # This should probably be a boolean True, not "true"
    "messages": [
        {
            "role": "user",
            "content": "Tell me a one paragraph story."
        },
        {
            "role": "assistant",
            "content": ""
        }
    ]
}
async def http_post_async(json_payload):

    # Send the POST request using httpx with streaming
    async with httpx.AsyncClient(timeout=30.0) as client:
        async with client.stream("POST", "http://localhost:12031/gen/v1/chat/completions", json=json_payload) as response:
            response.raise_for_status()
            async for chunk in response.aiter_text():  # Use aiter_text() to handle decoding
                chunk=json.loads(chunk)
                chunk=chunk["choices"][0]["delta"]["content"]
                if chunk:  # Check for non-empty chunks
                    print(chunk, end="", flush=True)

response=await http_post_async(json_payload)


Once upon a time, in a small village nestled between towering mountains, lived a young girl named Mia. She was known for her radiant smile and her love for nature. One day, while exploring the forest, she stumbled upon a wounded

### c) Test Tool Calling

In [3]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"What is the current time in Singapore\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"tools\": [\
            {\
                \"type\": \"function\",\
                \"function\": {\
                    \"name\": \"google\",\
                    \"description\": \"The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.\",\
                    \"parameters\": {\
                        \"type\": \"object\",\
                        \"properties\": {\
                            \"search_query\": {\
                                \"type\": \"string\",\
                                \"description\": \"The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively.\"\
                            }\
                        },\
                        \"required\": [\"search_query\"]\
                    }\
                }\
            }\
        ],\
        \"tool_choice\": \"required\"}"

{"id":"chatcmpl-ab20190d-6adb-4ece-adec-cda53b331c41","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":[{"id":"call_0559d0f2-fb13-458f-8925-d4179feef47a","function":{"arguments":"{\"search_query\": \"current time in Singapore\"}","name":"google"},"type":"function"}]}}],"created":1730253159,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":42,"prompt_tokens":354,"total_tokens":396,"completion_tokens_details":null,"prompt_tokens_details":null}}

### d) Test JSON Schema

In [4]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Foundation is a science fiction novel by American writer \
            Isaac Asimov. It is the first published in his Foundation Trilogy (later \
            expanded into the Foundation series). Foundation is a cycle of five \
            interrelated short stories, first published as a single book by Gnome Press \
            in 1951. Collectively they tell the early story of the Foundation, \
            an institute founded by psychohistorian Hari Seldon to preserve the best \
            of galactic civilization after the collapse of the Galactic Empire.\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"json_schema\": {\"properties\": \
            {\"title\": \
                {\"title\": \"Title\", \"type\": \"string\"}, \
                    \"summary\": {\"title\": \"Summary\", \"type\": \"string\"}, \
                    \"author\": {\"title\": \"Author\", \
                    \"type\": \"string\"\
                }, \
                \"published_year\": {\
                    \"title\": \"Published Year\", \
                    \"type\": \"integer\"}}, \
                \"required\": [\
                    \"title\", \
                    \"summary\", \
                    \"author\", \
                    \"published_year\"\
                ], \
                \"title\": \"Book\", \
                \"type\": \"object\"\
            },\
        \"tool_choice\": \"none\"}"

{"id":"chatcmpl-20888322-e430-42f7-a39b-ec58876e7477","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"{\n  \"title\": \"Foundation\",\n  \"summary\": \"Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.\",\n  \"author\": \"Isaac Asimov\",\n  \"published_year\": 1951\n }","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1730253175,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":148,"prompt_tokens":

### e) Shut down the API Service

---

## 5. Docker

This test should **NOT** be run in devcontainer.

**Instructions:** 

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: up**

**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: down**

#### Smoke Test

In [1]:
%%bash
curl http://localhost:12031


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    35  100    35    0     0    964      0 --:--:-- --:--:-- --:--:--  1060


{"message":"gai-ttt-svr-exllamav2"}

**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **Docker: stop**

### Debugging

a) Container must be started with "python -m debugpy --listen 0.0.0.0:5678 main.py"

b) Port 5678 must be opened.

c) Click on "Debug" in Tool bar

d) Select "Attach" > "Run and Debug"

e) Add a "breakpoint" in the code

f) Run the API test to see if it trigger the breakpoint.